# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
The data is already provided for you.  You'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.248091603053435
Number of lines: 4257
Average number of words in each line: 11.50434578341555

The sentences 0 to 10:
Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.




## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [3]:
import numpy as np
import problem_unittests as tests
from collections import Counter

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    if type(text) is not list:
        text = text.split()
    word_counts = Counter(text)
    sorted_words = sorted(word_counts, key=word_counts.get, reverse=True)
    int_to_vocab = {ii : word for ii, word in enumerate(sorted_words)}
    vocab_to_int = {word : ii for ii, word in int_to_vocab.items()}
    return (vocab_to_int, int_to_vocab)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [4]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    puncts = ['.', ',', '"', ';', '!', '?', '(', ')', '--', '\n']
    punct_value = ['||period||', '||comma||', '||quotation_mark||', '||semicolon||', '||exclamation_mark||', 
                  '||question_mark||', '||left_parantheses||', 'right_parantheses||', '||dash||', '||return||' ]
    punct_lookup = {}
    for i in range(len(puncts)):
        punct_lookup[puncts[i]] = punct_value[i]
    return punct_lookup
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [7]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.0.0
Default GPU Device: /gpu:0


### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following tuple `(Input, Targets, LearningRate)`

In [8]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    input = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    learning_rate = tf.placeholder(tf.float32, name='learning_rate')
    return (input, targets, learning_rate)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [9]:
def get_init_cell(batch_size, rnn_size, keep_prob=0.8):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    cell = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
    cell = tf.contrib.rnn.MultiRNNCell([lstm])
    cell_state = cell.zero_state(batch_size, tf.float32)
    initial_state = tf.identity(cell_state, name='initial_state')
    return (cell, initial_state)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [10]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    embedding = tf.Variable(tf.random_uniform((vocab_size, embed_dim), -1, 1))
    embed = tf.nn.embedding_lookup(embedding, input_data)
    return embed


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

Tests Passed


### Build RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [11]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    outputs, final_state = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)
    final_state = tf.identity(final_state, name='final_state')
    return (outputs, final_state)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [12]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    embedded_text = get_embed(input_data, vocab_size, embed_dim)
    outputs, final_state = build_rnn(cell, embedded_text)
    logits = tf.contrib.layers.fully_connected(
                inputs=outputs,
                num_outputs=vocab_size,
                activation_fn=None,
                weights_initializer=tf.truncated_normal_initializer(stddev=0.1),
                biases_initializer=tf.zeros_initializer()
                )
    
    return (logits, final_state)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

Tests Passed


### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

For exmple, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2], [ 7  8], [13 14]]
    # Batch of targets
    [[ 2  3], [ 8  9], [14 15]]
  ]

  # Second Batch
  [
    # Batch of Input
    [[ 3  4], [ 9 10], [15 16]]
    # Batch of targets
    [[ 4  5], [10 11], [16 17]]
  ]

  # Third Batch
  [
    # Batch of Input
    [[ 5  6], [11 12], [17 18]]
    # Batch of targets
    [[ 6  7], [12 13], [18  1]]
  ]
]
```

Notice that the last target value in the last batch is the first input value of the first batch. In this case, `1`. This is a common technique used when creating sequence batches, although it is rather unintuitive.

In [76]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: Thy length of sequence
    :return: Batches as a Numpy array
    """
    '''
    int_text = np.asarray(int_text)
    n_ints_batch = batch_size * seq_length
    n_batches = len(int_text)//n_ints_batch
    #Keep the ints to be used in batches
    int_text = int_text[: n_batches * n_ints_batch]
    #Reshape int_text
    int_text = int_text.reshape((batch_size, -1))
    all_batches = []
    for n in range(0, int_text.shape[1], seq_length):
        #batch features
        x = np.array(int_text[:, n: n+ seq_length])
        #labels placeholder
        y = np.array(x + 1)
        #Handles the wrap around of the last batch (Kind of Hacky, but it works)
        if n == int_text.shape[1] - seq_length:
            y[-1][-1] = int_text[0][0]
        all_batches.append([x, y])
    return np.array(all_batches)
    '''
    #Using logic provided by Udacity to create the batches.
    n_batches = int(len(int_text) / (batch_size * seq_length))

    # Drop the last few characters to make only full batches
    xdata = np.array(int_text[: n_batches * batch_size * seq_length])
    ydata = np.array(int_text[1: n_batches * batch_size * seq_length + 1])
    ydata[-1] = xdata[0]

    x_batches = np.split(xdata.reshape(batch_size, -1), n_batches, 1)
    y_batches = np.split(ydata.reshape(batch_size, -1), n_batches, 1)

    return np.array(list(zip(x_batches, y_batches)))

print(get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2))
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_batches(get_batches)

[array([[ 1,  2],
       [ 7,  8],
       [13, 14]]), array([[ 3,  4],
       [ 9, 10],
       [15, 16]]), array([[ 5,  6],
       [11, 12],
       [17, 18]])]
[[[[ 1  2]
   [ 7  8]
   [13 14]]

  [[ 2  3]
   [ 8  9]
   [14 15]]]


 [[[ 3  4]
   [ 9 10]
   [15 16]]

  [[ 4  5]
   [10 11]
   [16 17]]]


 [[[ 5  6]
   [11 12]
   [17 18]]

  [[ 6  7]
   [12 13]
   [18  1]]]]
[array([[   0,    1,    2,    3,    4],
       [  35,   36,   37,   38,   39],
       [  70,   71,   72,   73,   74],
       [ 105,  106,  107,  108,  109],
       [ 140,  141,  142,  143,  144],
       [ 175,  176,  177,  178,  179],
       [ 210,  211,  212,  213,  214],
       [ 245,  246,  247,  248,  249],
       [ 280,  281,  282,  283,  284],
       [ 315,  316,  317,  318,  319],
       [ 350,  351,  352,  353,  354],
       [ 385,  386,  387,  388,  389],
       [ 420,  421,  422,  423,  424],
       [ 455,  456,  457,  458,  459],
       [ 490,  491,  492,  493,  494],
       [ 525,  526,  527,  528,  529],


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [14]:
# Number of Epochs
num_epochs = 64
# Batch Size
batch_size = 128
# RNN Size
rnn_size = 256
# Embedding Dimension Size
embed_dim = 256
# Sequence Length
seq_length = 16
# Learning Rate
learning_rate = 0.01
# Show stats for every n number of batches
show_every_n_batches = 25

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

### Build the Graph
Build the graph using the neural network you implemented.

In [67]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    print(rnn_size, embed_dim)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')
    print('probs done')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))
    print('cost done')

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)
    print('optimizer done')

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    print('gradients done')
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)
    print('train op done')

256 256
probs done
cost done
optimizer done
gradients done
train op done


## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forums](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [68]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/33   train_loss = 8.833
Epoch   0 Batch   25/33   train_loss = 5.274
Epoch   1 Batch   17/33   train_loss = 4.595
Epoch   2 Batch    9/33   train_loss = 4.130
Epoch   3 Batch    1/33   train_loss = 3.684
Epoch   3 Batch   26/33   train_loss = 3.524
Epoch   4 Batch   18/33   train_loss = 3.225
Epoch   5 Batch   10/33   train_loss = 2.952
Epoch   6 Batch    2/33   train_loss = 2.721
Epoch   6 Batch   27/33   train_loss = 2.583
Epoch   7 Batch   19/33   train_loss = 2.265
Epoch   8 Batch   11/33   train_loss = 2.035
Epoch   9 Batch    3/33   train_loss = 1.937
Epoch   9 Batch   28/33   train_loss = 1.762
Epoch  10 Batch   20/33   train_loss = 1.669
Epoch  11 Batch   12/33   train_loss = 1.530
Epoch  12 Batch    4/33   train_loss = 1.485
Epoch  12 Batch   29/33   train_loss = 1.327
Epoch  13 Batch   21/33   train_loss = 1.166
Epoch  14 Batch   13/33   train_loss = 1.157
Epoch  15 Batch    5/33   train_loss = 1.006
Epoch  15 Batch   30/33   train_loss = 0.968
Epoch  16 

## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [69]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [70]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [71]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    input_tensor = loaded_graph.get_tensor_by_name('input:0')
    initial_state_tensor = loaded_graph.get_tensor_by_name('initial_state:0')
    final_state_tensor = loaded_graph.get_tensor_by_name('final_state:0')
    prob_tensor = loaded_graph.get_tensor_by_name('probs:0')
    return (input_tensor, initial_state_tensor, final_state_tensor, prob_tensor)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)

Tests Passed


### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [72]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    print(probabilities)
    idx = np.random.choice(np.arange(len(int_to_vocab)), p=probabilities)
    print(idx)
    return int_to_vocab[idx]


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)

[ 0.1   0.8   0.05  0.05]
1
Tests Passed


## Generate TV Script
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

In [73]:
gen_length = 200
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'moe_szyslak'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

[  2.46096347e-02   3.77040044e-09   1.50971886e-04 ...,   3.04071861e-08
   2.39860292e-08   4.53858853e-08]
3
[  1.31219605e-10   3.05740459e-12   1.28764270e-07 ...,   5.18824539e-09
   3.01153014e-09   6.23537799e-09]
3530
[  6.99917717e-13   2.04610498e-13   1.72553086e-07 ...,   6.23132101e-11
   4.95879310e-11   1.36859246e-10]
304
[  4.37196356e-11   7.20480208e-13   3.43256695e-12 ...,   1.39921273e-17
   8.13664018e-17   7.18653669e-16]
4
[  1.10244622e-08   3.54997355e-05   3.06587666e-09 ...,   1.17161905e-10
   1.42865525e-10   2.07103931e-10]
116
[  6.58526176e-08   7.77400537e-14   9.99996185e-01 ...,   1.22907420e-14
   2.05625583e-14   7.72331282e-14]
2
[  5.65042174e-05   1.22277806e-08   8.42281770e-06 ...,   2.73004449e-11
   1.62792890e-10   8.94696470e-11]
797
[  3.80064876e-06   6.01547256e-09   8.78050632e-04 ...,   5.78973610e-13
   4.86281588e-13   1.23085277e-12]
797
[  5.02438616e-06   1.75075510e-09   9.99688029e-01 ...,   8.55973126e-15
   5.32147179e-15  

[  6.56015158e-01   1.28465449e-09   1.62986089e-05 ...,   4.97046883e-11
   2.46066972e-10   2.60757027e-10]
0
[  6.30146649e-04   2.32568786e-01   6.03553829e-09 ...,   3.37034428e-11
   6.98630598e-11   2.63231853e-10]
27
[  1.18326079e-05   4.32882924e-10   9.97990489e-01 ...,   1.87144110e-13
   3.08657966e-13   1.72020688e-12]
2
[  8.90210117e-07   1.23489936e-05   1.90949850e-05 ...,   5.14274734e-10
   2.67865996e-09   5.22268051e-09]
47
[  6.90012588e-04   5.52965451e-11   4.41637298e-04 ...,   2.06191106e-11
   1.11123652e-10   3.40435874e-10]
11
[  2.58090068e-03   4.85995862e-11   1.15005219e-06 ...,   3.07091921e-11
   4.99597724e-11   1.67709902e-10]
170
[  2.81097554e-02   1.34842149e-09   8.11397040e-04 ...,   2.14972110e-10
   3.59045682e-10   6.96958768e-10]
13
[  3.23241557e-06   1.30361843e-06   1.13674196e-05 ...,   2.44473441e-10
   4.83008356e-10   4.96241603e-10]
95
[  3.21916975e-02   3.68180260e-08   5.17929379e-07 ...,   6.18648022e-10
   8.46551662e-10   5.4

[  3.49125185e-05   1.91692422e-08   1.74455578e-04 ...,   2.84211460e-10
   6.16314166e-10   7.96615052e-10]
44
[  5.67040503e-01   3.56895694e-11   3.90229225e-01 ...,   1.89508809e-09
   9.76667636e-10   2.05297113e-09]
0
[  1.13213295e-03   6.52343750e-01   2.69185296e-08 ...,   6.69229394e-11
   3.33123147e-11   1.01724851e-10]
75
[  7.06362130e-07   1.08195959e-08   2.92558484e-02 ...,   3.45646223e-09
   1.47293977e-09   5.32284705e-09]
377
[  7.10392856e-09   2.55084629e-08   3.73403211e-08 ...,   2.64480295e-11
   2.55978572e-11   5.20166375e-11]
250
[  3.19983868e-04   3.84706915e-08   2.33037355e-07 ...,   1.22205284e-11
   3.57602212e-11   5.37565339e-11]
42
[  4.88861522e-04   1.17506227e-10   6.80119399e-07 ...,   1.34599817e-10
   9.01928046e-11   2.76522444e-10]
4100
[  1.46740012e-05   2.37693511e-11   3.74804499e-09 ...,   1.07102520e-15
   5.36720225e-15   6.12462970e-15]
14
[  3.27022875e-09   6.25111594e-12   1.86585614e-09 ...,   1.87666861e-11
   1.16368859e-11  

moe_szyslak:(nasty laugh) ah, ha ha, you get the drinks!
lenny_leonard: geez, homer, this is, in a step i perhaps should have taken initially, let me look up the mr.(nervous chuckle)
homer_simpson:(out of breath, desperate) um, homer. i was thinking of another thing i got goin'.
the_rich_texan: gimme a" texas. homer, have a great to think of those two sits of a big girl ain't...
c. _montgomery_burns:(shaken) why, i think who can help wanted as the keys?
barney_gumble: from my dear, homer. i can't get drunk and come up to a moe_szyslak:...(under breath) losers.
lenny_leonard: it's official. maybe i'll have some fun.
moe_szyslak: i just wanna tell you all that even before i won this drink?
bart_simpson: no, o'reilly.
carl_carlson: they lost again, here. he must feel so helpless and scared.
homer_simpson: you mean like my car is me?
moe_szyslak:

# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckly there's more data!  As we mentioned in the begging of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.